- [get `map.json`](getmap.py)

In [80]:
import pandas as pd
import json

In [46]:
df=pd.read_csv("../../CY/Clean Management System/ManagementSystemItems.csv")
raw=pd.read_feather("raw.feather").drop(["Country","year","Tags"],axis=1)
raw.Q=raw.Q.str.replace("  "," ").str.lower().str.strip()
raw.columns=raw.columns.lower()

with open("map.json", "r") as j:
    hint=dict(json.loads(j.read()))

m13=df.columns[1:-2]
qid=df.QuestID.to_list()

In [52]:
# func.
def update(fast=True):
    global df
    filt=df.mngitems<13 if fast==True else df.mngitems<14
    df.loc[filt, "mngsum13items"]=df.loc[filt,m13].sum(axis=1, skipna=True)
    df.loc[filt,"mngitems"]=13-df.loc[filt,m13].isna().sum(axis=1)

def assign(qid, item, val):
    '''assign(y-axis,x-axis,val)'''
    global df
    assert (item:=item.lower().strip()) not in ["mngsum13items","mngitems"], f"use `update()` for {item} column."
    df.loc[df.QuestID==qid, item]=val

def kwSearch(kw:str):
    global raw
    return raw[raw.Q.str.contains(kw.lower())]

---

In [104]:
for i in hint:
    print(i)
    for j in range(len(hint[i])):
        temp=kwSearch(hint[i][j])
        print(f"└╴{hint[i][j]}: {len(temp)}")

hrsigned
└╴╴signed: 3152
└╴╴hr policy: 0
recruit
└╴╴recruitment: 17660
discfire
└╴╴disciplinary and termination: 0
grievdispute
└╴╴grievance handling: 0
└╴╴dispute resolution: 0
accounthr
└╴╴accountability: 64
└╴╴following hr policies: 0
└╴╴following factory policies and procedures: 0
comimplhr
└╴╴communicate and implement: 0
└╴╴implement hr: 0
investihr
└╴╴investigate: 186
└╴╴evaluate performance of hr: 0
└╴╴hr policies and procedures to identify weaknesses: 0
emergencepr
└╴╴emergency: 29806
└╴╴emergency preparedness: 0
riskcontr
└╴╴risk management: 64
accidentinvesti
└╴╴investigate: 186
└╴╴accident investigation: 0
accohs
└╴╴accountability: 64
└╴╴for carrying out: 64
impleosh
└╴╴implement osh: 0
└╴╴osh policies: 0
invstiosh
└╴╴investigate: 186
└╴╴measure osh issues: 0


In [106]:
kwSearch("investigate").Q.unique()

array(['does the employer investigate work-related accidents and indicate the technical recommendations necessary to prevent them?',
       'does the employer record and investigate work-related accidents and illnesses, and indicate the technical recommendations necessary to prevent them?',
       'does the employer record, investigate and report work-related accidents and diseases to the local manpower office?'],
      dtype=object)

## `accidentinvesti`

In [145]:
aci=raw[raw.Q.isin(['does the employer record and investigate work-related accidents and illnesses, and indicate the technical recommendations necessary to prevent them?',
       'does the employer record, investigate and report work-related accidents and diseases to the local manpower office?'])].QuestID.to_list()

for it in aci:
    if it in df[df.mngitems<13].QuestID.unique():
        print(it)
        break

42730


In [163]:
def match(qlist:list, item:str ,supress=True):
    global raw,df
    already=[]
    matched=[]
    tempRaw=raw[raw.Q.isin(qlist)]
    tempQID=tempRaw.QuestID.unique()

    for i in tempQID:
        if i in qid:
            if df.loc[df.QuestID==i, item].isna().values[0]==False:
                already.append(i)
            else:
                assign( i,item,int(tempRaw.loc[tempRaw.QuestID==i,"Finding"]) )
                matched.append(i)

    del tempRaw

    if supress==False:
        print(f"{len(matched)}/{len(tempQID)} successfully assigned.\n{len(already)}/{len(tempQID)} matched but already had value.")
        return matched

In [165]:
df[df.QuestID==42730]

,QuestID,hrsigned,recruit,discfire,grievdispute,accounthr,comimplhr,investihr,emergencepr,riskcontr,accidentinvesti,accohs,impleosh,invstiosh,mngsum13items,mngitems
32,42730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0


In [167]:
match(['does the employer record and investigate work-related accidents and illnesses, and indicate the technical recommendations necessary to prevent them?',
       'does the employer record, investigate and report work-related accidents and diseases to the local manpower office?']
       ,item="accidentinvesti")

0/184 successfully assigned.
184/184 matched but already had value.


[]